In [27]:
from pyspark.sql import DataFrameReader
from pyspark.sql import SparkSession
from pyspark.ml.feature import IndexToString, Normalizer, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from helpers.helper_functions import translate_to_file_string
from pyspark.sql.functions import col,lit,to_date
from pyspark.sql.functions import expr,when


# for pretty printing
def printDf(sprkDF): 
    newdf = sprkDF.toPandas()
    from IPython.display import display, HTML
    return HTML(newdf.to_html())

inputFile = translate_to_file_string("./data/RKI_COVID19_20210529.csv")

### Create Spark Session

In [42]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("RKICOVID19PREPARATION")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ",") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- ObjectId: integer (nullable = true)
 |-- IdBundesland: integer (nullable = true)
 |-- Bundesland: string (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- AnzahlFall: integer (nullable = true)
 |-- AnzahlTodesfall: integer (nullable = true)
 |-- Meldedatum: string (nullable = true)
 |-- IdLandkreis: integer (nullable = true)
 |-- Datenstand: string (nullable = true)
 |-- NeuerFall: integer (nullable = true)
 |-- NeuerTodesfall: integer (nullable = true)
 |-- Refdatum: string (nullable = true)
 |-- NeuGenesen: integer (nullable = true)
 |-- AnzahlGenesen: integer (nullable = true)
 |-- IstErkrankungsbeginn: integer (nullable = true)
 |-- Altersgruppe2: string (nullable = true)

None


In [20]:
df.count()

2003106

# Data Preperation

## Filtern von Korrekturwerten
Nimmt das Feld NeuerFall den Wert -1 an, so ist er laut RKI "[...] nur in der Publikation des Vortags enthalten". Das heißt es handelt sich folglich um eine Korrektur, der Puplikation des Vortages und muss in der aktuellen Datenauswertung näher betrachtet werden. Daraus ergibt sich, dass die entsprechenden Records aus dem Dataframe herausgefiltert werden müssen.

In [45]:
df = df.filter(df.NeuerFall > -1)
df.count()

KeyboardInterrupt: 

## Vereinzelung der Aggregierten der Werte
In der Spalte AnzahlFall steht jeweils die Summe der Fälle. Um nun die verschiedenen Modelle trainieren zu können, muss nun die Aggration rückgängig gemacht werden. Das heißt für jeden Fall muss nun ein Record im DataFrame aufgenommen werden. Die neue Anzahl der Records muss der Anzahl der gemeldeten Fälle entsprechen. Darüber hinaus ist eine neue Spalte anzufügen. Die neue Spalte gibt an ob die Person genesen, gestorben oder keins von beiden ist. Die Spalten AnzahlFall, AnzahlTodesfall und AnzahlGenesen können dann entfallen. Ebenfalls die Felder NeuerFall, NeuerTodesfall und NeuGenesen. 

### Vereinzelung

In [35]:
explodedDF = df.withColumn("AnzahlFall", expr("explode(array_repeat(AnzahlFall,int(AnzahlFall)))"))
explodedDF.count()

3675296

### Neue Spalte

In [36]:
explodedDFWithLabel = explodedDF.withColumn("FallStatus", when(explodedDF.AnzahlGenesen > 0, "GENESEN")
                                 .when(explodedDF.AnzahlTodesfall > 0, "GESTORBEN")
                                 .otherwise("NICHTEINGETRETEN"))

In [37]:
explodedDFWithLabel.show()

+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+----------+
|ObjectId|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|AnzahlFall|AnzahlTodesfall|          Meldedatum|IdLandkreis|          Datenstand|NeuerFall|NeuerTodesfall|            Refdatum|NeuGenesen|AnzahlGenesen|IstErkrankungsbeginn|    Altersgruppe2|FallStatus|
+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+----------+
|       1|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|         3|              0|2021/03/19 00:00:...|       1001|29.05.2021, 00:

### Löschen von unötigen Spalten

In [38]:
# definition der zu löschenden Spalten
columnsToDelete = ("Altersgruppe2", "AnzahlFall", "NeuerFall", "AnzahlTodesfall", "NeuerTodesfall", "AnzahlGenesen", "NeuGenesen")
dfWithDropedColumns = explodedDFWithLabel.drop(*columnsToDelete)

In [39]:
dfWithDropedColumns.show()

+--------+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+--------------------+--------------------+----------+
|ObjectId|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|          Meldedatum|IdLandkreis|          Datenstand|            Refdatum|IstErkrankungsbeginn|FallStatus|
+--------+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+--------------------+--------------------+----------+
|       1|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|29.05.2021, 00:00...|2021/03/16 00:00:...|                   1|   GENESEN|
|       1|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|29.05.2021, 00:00...|2021/03/16 00:00:...|                   1|   GENESEN|
|       1|           1|Schleswig-Holstein|SK Flensburg|     A15-A